# XSS

xss攻击也叫web注入.

实施XSS攻击需要具备两个条件：
1. 需要向web页面注入恶意代码；
2. 这些恶意代码能够被浏览器成功的执行。

解决办法：
1. 一种方法是在表单提交或者url参数传递前，对需要的参数进行过滤。
2. 在后台对从数据库获取的字符串数据进行过滤，判断关键字。
3. 设置安全机制。
django框架：内部机制默认阻止了。它会判定传入的字符串是不安全的，就不会渲染而以字符串的形式显示。如果手贱写了safe，那就危险了，若想使用safe,那就必须在后台对要渲染的字符串做过滤了。所以在开发的时候，一定要慎用安全机制。尤其是对用户可以提交的并能渲染的内容！！！如果使用make_safe() 标识 "alert(1)", 那么前端会弹出1的窗口，这显然不是正常用户希望看到的。

In [ ]:
def test(request):
    from django.utils.safestring import mark_safe
    temp = "<a href='http://www.baidu.com'>百度</a>"
    newtemp = mark_safe(temp)
    return render(request,'test.html',{'temp':newtemp}

# CSRF

CSRF, Cross Site Request Forgery, 跨站点请求伪造。举例来讲，某个恶意的网站上有一个指向你的网站的链接，如果某个用户已经登录到你的网站上了，那么当这个用户点击这个恶意网站上的那个链接时，就会向你的网站发来一个请求，你的网站会以为这个请求是用户自己发来的，其实呢，这个请求是那个恶意网站伪造的。

## Django 提供的 CSRF 防护机制

form表单中以加上 {% csrf_token %}：会做2件事，第一件是在cookie里加上 csrftoken ，第二件是在表单里加上一个 csrfmiddlewaretoken 的隐藏的input域。

在处理 POST 请求之前，django 会验证这个请求的 cookie 里的 csrftoken 字段的值和提交的表单里的 csrfmiddlewaretoken 字段的值(如果表单里没有就去请求头里找这个字段)是否一样。如果一样，则表明这是一个合法的请求，否则，这个请求可能是来自于别人的 csrf 攻击，返回 403 Forbidden.

## 前后端分离设置token

前端页面直接通过获取静态文件得到，然后直接发送ajax请求，得到csrftoken，此时需要一个视图函数来返回token值
```
def get_token(request):
    token = django.middleware.csrf.get_token(request)
    return JsonResponse({'token': token})
```
这个方法会自动给浏览器将token值保存在cookie中，前端发送ajax请求之前得到csrftoken后，将其添加到请求头中即可。



In [ ]:
def get_token(request):
    """
    Return the CSRF token required for a POST form. The token is an
    alphanumeric value. A new token is created if one is not already set.

    A side effect of calling this function is to make the csrf_protect
    decorator and the CsrfViewMiddleware add a CSRF cookie and a 'Vary: Cookie'
    header to the outgoing response.  For this reason, you may need to use this
    function lazily, as is done by the csrf context processor.
    """
    if "CSRF_COOKIE" not in request.META:
        csrf_secret = _get_new_csrf_string()
        request.META["CSRF_COOKIE"] = _salt_cipher_secret(csrf_secret)
    else:
        csrf_secret = _unsalt_cipher_token(request.META["CSRF_COOKIE"])
    request.META["CSRF_COOKIE_USED"] = True
    return _salt_cipher_secret(csrf_secret)

## django和csrf相关的两个装饰器

from django.views.decorators.csrf import csrf_exempt, csrf_protect

全站禁用：
    - 整个框架不使用csrf安全机制，直接在settings.py文件中注销，整个网站都不再应用。
        # 'django.middleware.csrf.CsrfViewMiddleware',
     
局部禁用：全局使用，但是某些函数不需要应用。
        - settings.py文件中不注销，在项目的views.py函数中导入模块，给函数或是类加上对应方法的装饰器：
        'django.middleware.csrf.CsrfViewMiddleware',
         
        from django.views.decorators.csrf import csrf_exempt,csrf_protect
 
        @csrf_exempt   #不再做检测！其他没加装饰器的函数还是会检测
        def csrf1(request):
 
            if request.method == 'GET':
                return render(request,'csrf1.html')
            else:
                return HttpResponse('ok')
     
局部使用：全局不使用，但是某些函数需要应用。
         
        # 'django.middleware.csrf.CsrfViewMiddleware',
         
        from django.views.decorators.csrf import csrf_exempt,csrf_protect
 
        @csrf_protect  #全站不用，某个函数需要使用认证的时候, 本质上也是走了CsrfViewMiddleware这个类的一些方法
        def csrf1(request):
 
            if request.method == 'GET':
                return render(request,'csrf1.html')
            else:
                return HttpResponse('ok')

In [ ]:
def csrf_exempt(view_func):
    """Mark a view function as being exempt from the CSRF view protection."""
    # view_func.csrf_exempt = True would also work, but decorators are nicer
    # if they don't have side effects, so return a new function.
    def wrapped_view(*args, **kwargs):
        return view_func(*args, **kwargs)
    # 给函数增加了一个属性值
    wrapped_view.csrf_exempt = True
    # 使用了wraps来保持原函数的属性
    return wraps(view_func)(wrapped_view)

## CBV 使用csrf认证


django 不能给类内的函数名上添加装饰器，只能是在类上添加。同时django限制若是应用装饰器，必须用它的方法去添加，同时添加的语法格式也有限制。

    from django.views import View
    from django.utils.decorators import method_decorator  #必须使用这个方法，@method_decorator(装饰器函数名称或方法,name='被装饰的函数名')

    #先导入方法，然后装饰器以参数的形式添加,其次指定要添加这个方法的函数名<样式：name="函数名">

    @method_decorator(csrf_protect,name='dispatch')
    class Foo(View):
        #请求来了，都是先执行类View的内置函数dispatch，然后再映射到对应的方法上！
        #所以给dispatch添加上就相当于给所有的方法添加了
        def get(self,request):
            pass

        def post(self,request):
            pass

PS：CBV中添加装饰器
    #自定义的装饰器
    def wrapper(func):
        def inner(self, request, *args,**kwargs):
            return func(self, request, *args,**kwargs)
        return inner

    # 1. 指定方法上添加装饰器
            class Foo(View):
                @method_decorator(wrapper)    #先导入方法，然后装饰器以参数的形式添加
                def get(self,request):
                    pass

                def post(self,request):
                    pass
    # 2. 在类上添加
            @method_decorator(wrapper,name='dispatch')
            class Foo(View):
                def get(self,request):
                    pass
                def post(self,request):
                    pass

## CsrfViewMiddleware

from django.middleware.csrf import CsrfViewMiddleware

In [ ]:
class CsrfViewMiddleware(MiddlewareMixin):
    """
    Require a present and correct csrfmiddlewaretoken for POST requests that
    have a CSRF cookie, and set an outgoing CSRF cookie.

    This middleware should be used in conjunction with the {% csrf_token %}
    template tag.
    """
    # The _accept and _reject methods currently only exist for the sake of the
    # requires_csrf_token decorator.
    def _accept(self, request):
        # 请求通过
        request.csrf_processing_done = True
        return None

    def _reject(self, request, reason):
        # 请求拒绝
        response = _get_failure_view()(request, reason=reason)
        log_response(
            'Forbidden (%s): %s', reason, request.path,
            response=response,
            request=request,
            logger=logger,
        )
        return response

    def _get_token(self, request):
        # 从cookie中去拿csrftoken的键值
        if settings.CSRF_USE_SESSIONS:
            try:
                return request.session.get(CSRF_SESSION_KEY)
            except AttributeError:
                raise ImproperlyConfigured(
                    'CSRF_USE_SESSIONS is enabled, but request.session is not '
                    'set. SessionMiddleware must appear before CsrfViewMiddleware '
                    'in MIDDLEWARE%s.' % ('_CLASSES' if settings.MIDDLEWARE is None else '')
                )
        else:
            try:
                cookie_token = request.COOKIES[settings.CSRF_COOKIE_NAME]
            except KeyError:
                return None

            csrf_token = _sanitize_token(cookie_token)
            if csrf_token != cookie_token:
                # Cookie token needed to be replaced;
                # the cookie needs to be reset.
                request.csrf_cookie_needs_reset = True
            return csrf_token

    def _set_token(self, request, response):
        if settings.CSRF_USE_SESSIONS:
            request.session[CSRF_SESSION_KEY] = request.META['CSRF_COOKIE']
        else:
            response.set_cookie(
                settings.CSRF_COOKIE_NAME,
                request.META['CSRF_COOKIE'],
                max_age=settings.CSRF_COOKIE_AGE,
                domain=settings.CSRF_COOKIE_DOMAIN,
                path=settings.CSRF_COOKIE_PATH,
                secure=settings.CSRF_COOKIE_SECURE,
                httponly=settings.CSRF_COOKIE_HTTPONLY,
                samesite=settings.CSRF_COOKIE_SAMESITE,
            )
            # Set the Vary header since content varies with the CSRF cookie.
            patch_vary_headers(response, ('Cookie',))

    def process_request(self, request):
        csrf_token = self._get_token(request)
        if csrf_token is not None:
            # Use same token next time.
            # 往request.META里存一下，接下来的请求都可以用到这个里面的东西了
            request.META['CSRF_COOKIE'] = csrf_token

    def process_view(self, request, callback, callback_args, callback_kwargs):
        if getattr(request, 'csrf_processing_done', False):
            return None

        # Wait until request.META["CSRF_COOKIE"] has been manipulated before
        # bailing out, so that get_token still works
        # 使用csrf_exempt就直接过
        if getattr(callback, 'csrf_exempt', False):
            return None

        # Assume that anything not defined as 'safe' by RFC7231 needs protection
        if request.method not in ('GET', 'HEAD', 'OPTIONS', 'TRACE'):
            if getattr(request, '_dont_enforce_csrf_checks', False):
                return self._accept(request)

            if request.is_secure():
                # HTTPS 相关
                ...

            csrf_token = request.META.get('CSRF_COOKIE')
            if csrf_token is None:
                # No CSRF cookie. For POST requests, we insist on a CSRF cookie,
                # and in this way we can avoid all CSRF attacks, including login
                # CSRF.
                # post 请求里竟然没有到csrftoken的cookie，直接拒绝掉
                return self._reject(request, REASON_NO_CSRF_COOKIE)

            # Check non-cookie token for match.
            request_csrf_token = ""
            if request.method == "POST":
                try:
                    request_csrf_token = request.POST.get('csrfmiddlewaretoken', '')
                except IOError:
                    pass

            if request_csrf_token == "":
                # Fall back to X-CSRFToken, to make things easier for AJAX,
                # and possible for PUT/DELETE.
                request_csrf_token = request.META.get(settings.CSRF_HEADER_NAME, '')
            # 这里先去request.POST里取, 如果此时请求体的content-type不是urlencoded，那么django就不会帮我们解析数据放在request.POST里
            # 比如ajax发送的数据content-type 是 json格式，那么request.POST就没有相应的数据，此时就去请求头找，所以ajax请求带头X-CSRFTOKEN
            request_csrf_token = _sanitize_token(request_csrf_token)
            if not _compare_salted_tokens(request_csrf_token, csrf_token):
                # 不相等直接拒绝
                return self._reject(request, REASON_BAD_TOKEN)

        return self._accept(request)

    def process_response(self, request, response):
        if not getattr(request, 'csrf_cookie_needs_reset', False):
            if getattr(response, 'csrf_cookie_set', False):
                return response

        if not request.META.get("CSRF_COOKIE_USED", False):
            return response

        # Set the CSRF cookie even if it's already set, so we renew
        # the expiry timer.
        # 设置token
        self._set_token(request, response)
        response.csrf_cookie_set = True
        return response